In [55]:
import os
from game import PokerGame
from player import PlayerAction
from baseplayers import *
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.prompts import ChatPromptTemplate
from operator import itemgetter
import getpass
from langchain_mistralai import ChatMistralAI

In [56]:
# os.environ['LANGCHAIN_TRACING_V2'] = 'false'
# os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
# os.environ["LANGSMITH_API_KEY"] = getpass.getpass("Enter your LangSmith API key: ")
os.environ["MISTRAL_API_KEY"] = getpass.getpass("Enter your Mistral API key: ")

In [57]:
llm = ChatMistralAI(model="mistral-large-latest", temperature=0)

In [67]:
def card_from_index(index) -> str:
    if index == 0:
        return "XX"
    index -= 1  # since it is 1-indexed
    suit = index // 13
    rank = index % 13
    res = ''
    if rank == 12:
        res += 'A'
    elif rank == 11:
        res += 'K'
    elif rank == 10:
        res += 'Q'
    elif rank == 9:
        res += 'J'
    else:
        res += str(rank + 2)

    if suit == 0:
        res += 'S'
    elif suit == 1:
        res += 'H'
    elif suit == 2:
        res += 'D'
    else:
        res += 'C'

    return res

In [79]:
template = """You are a professional poker player. You will be given the current poker game state as a sequence of numbers
        structured as follows.
        [hole card 1, hole card 2, community card 1, community card 2, community card 3, community card 4, community card 5,
        pot, current raise amount, number of players, stack of player 1, stack of player 2, ... , stack of player n, blind amount,
        game number]\n
        The cards will be given as a string of two characters, with the rank followed by the suit. The string "XX" indicates that the card has not been revealed yet. The current raise amount includes the big blind.\n
        Given the game state, you have to make a move in the game. You have to decide the action and the amount you will put in the pot. You will either call, raise or fold. Consider check as call with amount 0. The amount will be 0 should you choose to fold.
        \nYou must provide only the action and the amount, and nothing else.
        \n\nHere is the game state, as a list of numbers:
        {game_state}
        \nYou are player number {player_number}
        \n\nYour output must be in the format:\nACTION, AMOUNT
        """

prompt = ChatPromptTemplate.from_template(template)

In [80]:
players = [
        InputPlayer("Alice", 1000),
        InputPlayer("Bob", 1000),
        InputPlayer("Charlie", 1000),
        InputPlayer("David", 1000),
        LLMPlayer("Eve", 1000)
    ]

# Create game
game = PokerGame(players, big_blind=20)
game.start_new_hand()


====== NEW HAND ======
Charlie posts big blind: 20

Phase: pre-flop
Pot: 20

Players:
  Alice : $1000 [hidden] active
  Bob (BTN): $1000 [hidden] active
  Charlie : $980 [hidden] active
→ David : $1000 [hidden] active
  Eve : $1000 [hidden] active


In [81]:
chain = prompt | llm

In [82]:
game_state = game.get_game_state()
game_state[:7] = [card_from_index(i) for i in game_state[:7]]
output = chain.invoke({'game_state': game_state, 'player_number': game.active_player_index + 1})
print(output.content)

CALL, 0


In [83]:
game.player_action(PlayerAction.CALL, 20)

David calls 20

Phase: pre-flop
Pot: 40

Players:
  Alice : $1000 [hidden] active
  Bob (BTN): $1000 [hidden] active
  Charlie : $980 [hidden] active
  David : $980 [hidden] active
→ Eve : $1000 [hidden] active


True

In [50]:
output = chain.invoke({'game_state': game.get_game_state(), 'player_number': game.active_player_index + 1})
print(output.content)

RAISE,40


In [51]:
game.player_action(PlayerAction.RAISE, 40)

Eve raises 40

Phase: pre-flop
Pot: 80

Players:
→ Alice : $1000 [hidden] active
  Bob (BTN): $1000 [hidden] active
  Charlie : $980 [hidden] active
  David : $980 [hidden] active
  Eve : $960 [hidden] active


True

In [52]:
output = chain.invoke({'game_state': game.get_game_state(), 'player_number': game.active_player_index + 1})
print(output.content)

RAISE, 80


In [53]:
game.player_action(PlayerAction.RAISE, 80)

Alice raises 80

Phase: pre-flop
Pot: 160

Players:
  Alice : $920 [hidden] active
→ Bob (BTN): $1000 [hidden] active
  Charlie : $980 [hidden] active
  David : $980 [hidden] active
  Eve : $960 [hidden] active


True

In [54]:
output = chain.invoke({'game_state': game.get_game_state(), 'player_number': game.active_player_index + 1})
print(output.content)

Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..


ResourceExhausted: 429 Resource has been exhausted (e.g. check quota).

In [37]:
game.player_action(PlayerAction.RAISE, 240)

Bob raises 240

Phase: pre-flop
Pot: 400

Players:
  Alice : $920 [hidden] active
  Bob (BTN): $760 [hidden] active
→ Charlie : $980 [hidden] active
  David : $980 [hidden] active
  Eve : $960 [hidden] active


True